# Base model evaluation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import re
from dateutil.parser import parse
import string
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import missingno as msno
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import scipy
import researchpy as rp
from matplotlib.offsetbox import AnchoredText
import pacmap
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
import pickle
import pmdarima as pm
from ipywidgets import Button, HBox, VBox
from ipywidgets import interactive
import ipywidgets
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
import xgboost as xgb
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [2]:
# read df full csv
df_full=pd.read_csv('../../data/consolidated data/df_full_01November2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')
df_full.head(1)

,sag mill expert system,Limitado por Límite Alto de Tonelaje,Optimizando tonelaje,Limitado por Peso,Limitado por Corriente Bobina,Limitado por torque espesador 2,Limitado por torque espesador 3,Limitado por Potencia Bolas 1,Limitado por Potencia Bolas 2,Limitado por Pebbles,Limitado por Corriente harnero,Limitado por Potencia SAG,Limitado por limite bajo de tonelaje,Limitado por Temperatura bobina,Limitado por presión espesador 2,Limitado por presión espesador 3,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,torque,charge cell,power,TPH,solid percentage,water,speed,solid,LL charge cell,HH charge cell,HH TPH,stator current,granulometry,Edad,dif_TPH_HH,dif_CC_HH,dif_CC_LL,loss of TPH,cluster,recommendation,loss of TPH_1,loss of TPH_2,loss of TPH_3,loss of TPH_4,loss of TPH_5,greater than HH CC,greater than HH CC_1,greater than HH CC_2,greater than HH CC_3,greater than HH CC_4,greater than HH CC_5,less than LL CC,less than LL CC_1,less than LL CC_2,less than LL CC_3,less than LL CC_4,less than LL CC_5,greater than Rec HH CC,greater than Rec HH CC_1,greater than Rec HH CC_2,greater than Rec HH CC_3,greater than Rec HH CC_4,greater than Rec HH CC_5,Rec HH CC less than HH CC,Rec HH CC less than HH CC_1,Rec HH CC less than HH CC_2,Rec HH CC less than HH CC_3,Rec HH CC less than HH CC_4,Rec HH CC less than HH CC_5,start loss tph,end loss tph,loss of TPH_mod
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-10 04:05:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.76,0.5,0.02,19.7649,98.5569,15.4504,0.05,64.384706,809.23733,22008.766259,4317.030837,73.888038,1539.203093,9.063024,73.155012,770.0,850.0,3950.0,3755.398344,48.829422,1.0,-367.030837,40.76267,39.23733,0,2,892.044136,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [3]:
list_df=[]
for i in df_full[df_full["start loss tph"]==1].index:
    try:
        #i="2022-02-27 12:31:00"
        inicio_perdida=i
        test=df_full[inicio_perdida:inicio_perdida+timedelta(hours=5)]
        fin_perdida=min(test[(test["end loss tph"]==1)].index)+timedelta(minutes=1)
        inicio_evento=inicio_perdida-timedelta(minutes=5)
        data=df_full.copy()[inicio_evento:fin_perdida]
        #display(data)
        dic_df={"inicio evento":inicio_evento,"inicio perdida":inicio_perdida,"fin perdida":fin_perdida,"variación HH TPH":data["HH TPH"].std()!=0,"Actua sistema de control":any(data["greater than HH CC"]==1),
        "Supera recomendación del modelo":any(data["greater than Rec HH CC"]==1),"Recomendación menor a HH CC":any(data["Rec HH CC less than HH CC"]==1),"CC menor a LL CC":any(data["less than LL CC"]==1),
        "Causalidad":any(data[data["greater than Rec HH CC"]==1].index<=inicio_perdida)}
        list_df.append(pd.DataFrame(dic_df,index=[1]))  
    except:
        #print(i)
        pass

In [4]:
df_tph_loss=pd.concat(list_df).reset_index(drop=True)
df_tph_loss.head()

,inicio evento,inicio perdida,fin perdida,variación HH TPH,Actua sistema de control,Supera recomendación del modelo,Recomendación menor a HH CC,CC menor a LL CC,Causalidad
0,2020-01-10 05:23:00,2020-01-10 05:28:00,2020-01-10 05:35:00,True,False,False,False,False,False
1,2020-01-10 07:05:00,2020-01-10 07:10:00,2020-01-10 08:41:00,False,True,False,False,False,False
2,2020-01-10 09:05:00,2020-01-10 09:10:00,2020-01-10 10:16:00,False,True,False,False,False,False
3,2020-01-10 10:15:00,2020-01-10 10:20:00,2020-01-10 11:08:00,False,True,True,True,False,False
4,2020-01-14 03:54:00,2020-01-14 03:59:00,2020-01-14 04:31:00,False,True,False,False,False,False


# Perdida de TPH por superar recomendación [+]

In [5]:
evento1=df_tph_loss[(df_tph_loss["Actua sistema de control"]==False) & (df_tph_loss["variación HH TPH"]==False) & (df_tph_loss["CC menor a LL CC"]==False) & (df_tph_loss["Recomendación menor a HH CC"]==True) \
    & (df_tph_loss["Supera recomendación del modelo"]==True)\
    & (df_tph_loss["Causalidad"]==True)].reset_index(drop=True)#.sample(1)
evento1

,inicio evento,inicio perdida,fin perdida,variación HH TPH,Actua sistema de control,Supera recomendación del modelo,Recomendación menor a HH CC,CC menor a LL CC,Causalidad
0,2020-03-05 11:24:00,2020-03-05 11:29:00,2020-03-05 11:44:00,False,False,True,True,False,True
1,2020-03-05 15:41:00,2020-03-05 15:46:00,2020-03-05 15:52:00,False,False,True,True,False,True
2,2020-03-10 02:19:00,2020-03-10 02:24:00,2020-03-10 02:32:00,False,False,True,True,False,True
3,2020-03-15 01:35:00,2020-03-15 01:40:00,2020-03-15 02:10:00,False,False,True,True,False,True
4,2020-03-15 13:41:00,2020-03-15 13:46:00,2020-03-15 13:54:00,False,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...
226,2022-02-26 16:42:00,2022-02-26 16:47:00,2022-02-26 16:53:00,False,False,True,True,False,True
227,2022-02-27 04:51:00,2022-02-27 04:56:00,2022-02-27 05:21:00,False,False,True,True,False,True
228,2022-02-27 05:29:00,2022-02-27 05:34:00,2022-02-27 05:40:00,False,False,True,True,False,True
229,2022-02-27 05:46:00,2022-02-27 05:51:00,2022-02-27 06:48:00,False,False,True,True,False,True


In [18]:
list_loss_tph=[]
for i in range(len(evento1)):
    data=df_full[evento1.iloc[i]["inicio perdida"]:evento1.iloc[i]["fin perdida"]]
    list_loss_tph.append(data["dif_TPH_HH"].sum())
tph1=sum(list_loss_tph)
tph1   

360145.1426394929

# Perdida de TPH sin superar recomendación [-]

In [6]:
evento2=df_tph_loss[(df_tph_loss["Actua sistema de control"]==False) & (df_tph_loss["variación HH TPH"]==False) & (df_tph_loss["CC menor a LL CC"]==False) & (df_tph_loss["Recomendación menor a HH CC"]==False)\
     & (df_tph_loss["Supera recomendación del modelo"]==False)\
    ].reset_index(drop=True)
evento2

,inicio evento,inicio perdida,fin perdida,variación HH TPH,Actua sistema de control,Supera recomendación del modelo,Recomendación menor a HH CC,CC menor a LL CC,Causalidad
0,2020-02-05 10:33:00,2020-02-05 10:38:00,2020-02-05 10:44:00,False,False,False,False,False,False
1,2020-02-05 11:30:00,2020-02-05 11:35:00,2020-02-05 11:45:00,False,False,False,False,False,False
2,2020-02-06 09:04:00,2020-02-06 09:09:00,2020-02-06 09:41:00,False,False,False,False,False,False
3,2020-02-06 11:54:00,2020-02-06 11:59:00,2020-02-06 12:07:00,False,False,False,False,False,False
4,2020-02-06 12:57:00,2020-02-06 13:02:00,2020-02-06 13:08:00,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
775,2022-01-15 12:37:00,2022-01-15 12:42:00,2022-01-15 12:50:00,False,False,False,False,False,False
776,2022-01-15 19:29:00,2022-01-15 19:34:00,2022-01-15 19:43:00,False,False,False,False,False,False
777,2022-01-15 21:45:00,2022-01-15 21:50:00,2022-01-15 21:56:00,False,False,False,False,False,False
778,2022-01-16 02:06:00,2022-01-16 02:11:00,2022-01-16 02:20:00,False,False,False,False,False,False


In [19]:
list_loss_tph=[]
for i in range(len(evento2)):
    data=df_full[evento2.iloc[i]["inicio perdida"]:evento2.iloc[i]["fin perdida"]]
    list_loss_tph.append(data["dif_TPH_HH"].sum())
tph2=sum(list_loss_tph)
tph2  

1635822.1399881667

# Costo de oportunidad TPH --> se supera recomendación pero aun así no hay caida de TPH [-]

In [3]:
# Desfase de perdida de TPH para eliminar las causas de las perdidas
df_cost=df_full.copy()
for i in range(1,6):
    df_cost[f"loss of TPH_{i}"]=df_cost["loss of TPH"].shift(-i)

df_cost["loss of TPH"]=df_cost.apply(lambda x: 1 if (x['loss of TPH']==1) or (x['loss of TPH_1']==1) or \
(x['loss of TPH_2']==1) or (x['loss of TPH_3']==1) or (x['loss of TPH_4']==1) or (x['loss of TPH_5']==1) else 0,axis=1)

In [4]:
# TPH promedio según HH charge cell
TPH_historico=df_cost[["HH charge cell","TPH"]].groupby("HH charge cell").mean().reset_index()
TPH_historico

,HH charge cell,TPH
0,660.000000,4349.634608
1,660.002292,4295.012475
2,660.004585,4294.244128
3,660.006877,4304.960976
4,660.009169,4307.145630
...,...,...
2971,1012.710324,4058.039942
2972,1012.780870,4066.605017
2973,1012.851417,4029.256336
2974,1012.921963,4015.668759


In [6]:
N = 1000
t = np.linspace(0, 10, 100)
y = np.sin(t)

fig = go.Figure(data=go.Scatter(x=TPH_historico["HH charge cell"], y=TPH_historico["TPH"], mode='lines'))

fig.update_layout(height=500, width=1200, title_text="Celda de carga vs TPH")
fig.update_layout(hovermode="x unified")
#

fig.show()

In [22]:
def cost_TPH(df,cc):
    df["distance"]=(df["HH charge cell"]-cc).abs()
    x=df[df["distance"]==df.distance.min()]["TPH"].values[0]
    return x

In [23]:
df=df_cost[(df_cost["loss of TPH"]==0) & (df_cost["Rec HH CC less than HH CC"]==1) & (df_cost["greater than Rec HH CC"]==1)]
df.head(1)

,sag mill expert system,Limitado por Límite Alto de Tonelaje,Optimizando tonelaje,Limitado por Peso,Limitado por Corriente Bobina,Limitado por torque espesador 2,Limitado por torque espesador 3,Limitado por Potencia Bolas 1,Limitado por Potencia Bolas 2,Limitado por Pebbles,Limitado por Corriente harnero,Limitado por Potencia SAG,Limitado por limite bajo de tonelaje,Limitado por Temperatura bobina,Limitado por presión espesador 2,Limitado por presión espesador 3,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,torque,charge cell,power,TPH,solid percentage,water,speed,solid,LL charge cell,HH charge cell,HH TPH,stator current,granulometry,Edad,dif_TPH_HH,dif_CC_HH,dif_CC_LL,loss of TPH,cluster,recommendation,loss of TPH_1,loss of TPH_2,loss of TPH_3,loss of TPH_4,loss of TPH_5,greater than HH CC,greater than HH CC_1,greater than HH CC_2,greater than HH CC_3,greater than HH CC_4,greater than HH CC_5,less than LL CC,less than LL CC_1,less than LL CC_2,less than LL CC_3,less than LL CC_4,less than LL CC_5,greater than Rec HH CC,greater than Rec HH CC_1,greater than Rec HH CC_2,greater than Rec HH CC_3,greater than Rec HH CC_4,greater than Rec HH CC_5,Rec HH CC less than HH CC,Rec HH CC less than HH CC_1,Rec HH CC less than HH CC_2,Rec HH CC less than HH CC_3,Rec HH CC less than HH CC_4,Rec HH CC less than HH CC_5,start loss tph,end loss tph,loss of TPH_mod
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-02-08 16:24:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.51,1.65,0.39,0.01,19.8478,112.1727,15.7825,0.03,63.117787,787.21956,19641.296793,4079.304899,73.877412,1439.921189,8.249531,74.0,720.0,800.0,4100.0,3667.648521,66.705534,2.0,20.695101,12.78044,67.21956,0,0,774.588893,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0,1.0,1.0,1.0,0,0,0


In [24]:
df_metric=df[["charge cell","TPH","HH TPH","recommendation","HH charge cell"]]
df_metric["TPH historic"]=df["recommendation"].apply(lambda x: cost_TPH(TPH_historico,x))
df_metric["cost TPH"]=df_metric["TPH"]-df_metric["TPH historic"]
df_metric

,charge cell,TPH,HH TPH,recommendation,HH charge cell,TPH historic,cost TPH
Timestamp,,,,,,,
2020-02-08 16:24:00,787.219560,4079.304899,4100.0,774.588893,800.0,3989.669743,89.635156
2020-02-08 16:25:00,787.558897,4087.649222,4100.0,774.588893,800.0,3989.669743,97.979479
2020-02-08 16:26:00,788.579123,4080.193127,4100.0,774.588893,800.0,3989.669743,90.523384
2020-02-08 16:27:00,790.698064,4093.222860,4100.0,774.588893,800.0,3989.669743,103.553117
2020-02-08 16:28:00,791.900323,4084.807274,4100.0,774.588893,800.0,3989.669743,95.137531
...,...,...,...,...,...,...,...
2022-02-28 09:54:00,914.137054,4100.673792,4120.0,907.966856,950.0,3803.968354,296.705438
2022-02-28 09:55:00,919.262837,4090.384509,4120.0,907.966856,950.0,3803.968354,286.416155
2022-02-28 09:56:00,922.912778,4082.137827,4120.0,918.208691,950.0,3778.918581,303.219246


In [25]:
costo_TPH=df_metric["cost TPH"].sum()
costo_TPH

7829440.7406747155

# TPH ganado!

In [31]:
tph1-tph2-costo_TPH

-9105117.73802339